<div class="alert alert-info">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Привет! Меня зовут Алина Гусева, я буду делать ревью к твоей работе по предобработке данных. В дальнейшем предлагаю общаться на "ты".
    
    
Видно, что ты старался, делая проект. Все ключевые этапы проекта выполнены, в целом тебе удалось со всем справиться. Особенно хочется отметить, что соблюдена структура проекта, а код написан аккуратно.
    
Для того, чтобы и тебе и мне удобнее бы
    ло ориентироваться в комментариях, я буду помечать их разными цветами:
- красный цвет означает комментарии, которые требуют доработки. Как только ты их исправишь - проект будет принят;
- желтый цвет - то, что в следующий раз можно будет сделать по-другому. На твое усмотрение можешь доделать маленькие правки сейчас или учесть замечание для выполнения следующих проектов;
- зеленый цвет - очень элегантные и удачные решение, которые желательно использовать в дальнейшей работе.
- синий цвет - информация
    
Предлагаю работать над проектом в диалоге: если ты что-то изменяешь или вносишь какие-то коррективы - выделяй это каким-нибудь цветом и пиши об этом мне. Так мне гораздо легче будет отследить сделанные изменения.
    
Также попрошу тебя не изменять и не удалять мои комментарии, чтобы проверка твоего самостоятельного проекта прошла оперативнее.
    
 
</div>

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования:** 
- Проверка гипотезы: на надежность клиента банка влияет наличие у него детей.
- Проверка гипотезы: на надежность клиента банка влияет семеное положение

## Шаг 1. Откройте файл с данными и изучите общую информацию

## Предварительное изучение

In [1]:
import pandas as pd
df = pd.read_csv('data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
df['income_type'].value_counts()  #бросилось в глаза "сотрудник", решил посмотреть это что такое

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [4]:
df.columns  #Проверка на кривизну названий столбцов

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [5]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [6]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [7]:
df['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [8]:
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [9]:
df['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [10]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

In [11]:
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [12]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [13]:
df.sort_values(by=['total_income'], ascending=False) #чтоб не копировать код, тут же проверял и минимальные

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12412,0,-1477.438114,44,высшее,0,женат / замужем,0,M,компаньон,0,2.265604e+06,ремонт жилью
19606,1,-2577.664662,39,высшее,0,женат / замужем,0,M,компаньон,1,2.200852e+06,строительство недвижимости
9169,1,-5248.554336,35,среднее,1,гражданский брак,1,M,сотрудник,0,1.726276e+06,дополнительное образование
20809,0,-4719.273476,61,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1.715018e+06,покупка жилья для семьи
17178,0,-5734.127087,42,высшее,0,гражданский брак,1,M,компаньон,0,1.711309e+06,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [14]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Вывод**  
- `days_employed` Не очень понятно что с графой трудовой стаж и откуда растут ноги... по идеи значение должно быть int в днях... а сейчас какая то билиберда(ушл к преподу). Ну еще пропуски есть... 
- `income_type` Не понятен статус "сотрудник", если сотрудние банка: то надо его к черту вычеркнуть... 
- `children` 20детей... надо будет глянуть на рекордсмена и либо удалить либо оставить если это женщина из Воронежа. -1 тоже так себе показатель... может беременных занесли или еще чего.
- `education` Образование надо чистить - разные регисторы, может и орфографические ошибки.
- `education_id` Надо сравнить с education ... Если все соответсвует, то просто столбец со str значением удалить.
- `dob_years` По хорошему надо сгруппировать не несколько статусов.Также есть новорожденные по возрасту... надо посмотреть сколько их, если мало удалить, если много - то куда то распихать
- `total_income` есть пропуск зарплат... не очень понятно пока что с этим делать. и формат еще экспоненциальный "Эх... Чтоб у меня зарпалата также записывалась( 
- `family_status` и `family_status_id` внешне все хорошо при том что одни из самых важных ячеек в нашем исследовании
- `debt` Вроде все хорошо.
- `purpose` цель кредита: не уверен что вообще столбец нужен для поставленной задачи исследований. Разве что факультативно.

удивительно но столбцы названы корректно...  
**Вопрос ревьюверу: изучая данные, смотрю уникальность значений, надо ли это отображать вообще в тетрадке?**


<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Молодец, хороший тон использовать метод info() для получения общей информации из таблицы. Для более детального изучения можно воспользоваться методом describe(). 
    
Информация никогда не бывает лишней. По заданию отображать уникальные значения не требуется, но попрактиковаться можно) Если ты делаешь одно и то же действие несколько раз, то удобнее написать цикл и прогнать через него все необходимые столбцы. Так будет гораздо удобнее.   

 
</div>

## Шаг 2. Предобработка данных

Пообщавшись с преподавателем, сделали вывод, что "сотрудник" и "компаньон" в первом слуае подразумевается "рядовой сотрудник какой то компании" "компаньон" - учредитель живущий диведенты. Данные по зарплате будем заполнять по медиане для соответствующего "типа занятости", т.к. может быть большой разброс. как мы увидели при изучении, есть очень хорошие зарплаты, которые могут попортить среднеарифметическое. Итого надо найти медиану для каждого типа занятости, затем заполнить пропуски. также стоит посмотреть не закрались ли отрицательные зарплаты.

### Обработка пропусков

In [15]:
df[df['total_income'] < 0]  # не затисались нули в зарплаты... будем верить полученным данным

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [16]:
df[df['total_income'].isna()].head(5) # просто посмотрел у кого пропуски... пропуски случайные(или не вижу закономерности, делал сначала head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Пропуски случайны в одном столбце. А если посмотреть пропуски в другом? В одних и тех же строках они находятся? Если да, то можно предположить, что произошел какой-нибудь сбой в системе
 
</div>

In [17]:

df_mediana_income = df.groupby('income_type')['total_income'].median()  # Делаем df с медианными значениями

In [18]:
def change_income(row):
    '''Check income, if NoneType in ['total_income'] return mediana from dataFrame with mediana value, else return total_income
    Args:
        row (row): just row, which fuction check
    Returns:
        float: income
    '''
    if pd.isna(row['total_income']):
        return df_mediana_income.loc[row['income_type']]
    return row['total_income']


<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Логично, что уровень дохода зависит от типа занятости. Здесь можно делать замену через цикл, или же воспользоваться очень удобнвм методом <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.transform.html'>transofrm()</a>. Решение получается почти в одну строчку
</div>

In [19]:
columns_name = ['income_type', 'total_income']  # Проверка работоспособности функции
values = [['безработный', 32432.324],['госслужащий', ]]
test_func_df = pd.DataFrame(data=values, columns=columns_name)
test_func_df['total_income'] = test_func_df.apply(change_income, axis=1)
test_func_df 

,income_type,total_income
0,безработный,32432.324000
1,госслужащий,150447.935283


In [20]:
df['total_income'] = df.apply(change_income, axis=1)
df.isna().sum() # Проверяем пропуски в зарплатах

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

In [21]:
# Будем надеятся что числа совпадают, знаки рандомны. попробую в "стаже" просто попробую привести к читабельному виду и к годам
def preproccesing_age(days_employed):
    '''function to try preprocessing empoyed

    Args:
        age (float): days_employed

    Returns:
        float: correct age in years
    '''
    if days_employed < 0.0:
        days_employed *= -1.0
    days_employed /= 365.25
    return days_employed


# print(preproccesing_age(-1245.123)) # Проверка на жизнеспособность функции
df['days_employed'] = df['days_employed'].apply(preproccesing_age)

<div class="alert alert-info">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Чтобы избавиться от отрицательных значений, можно воспользоваться методом abs(), который возвращает модуль числа
</div>

In [22]:
df['days_employed'] = df['days_employed'].fillna(0)  
df.isna().sum() 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
В столбце со стажем много некорректных значений. И даже заменив отрицательные значения и слишком большие значения, проблема с пропусками никуда не исчезнет. Но в данном случае можно пропуски заменить на 0, поскольку для дальнейшего исследования этот столбец нам не нужен. Но обязательно сообщить разработчикам базы данных о подобной проблеме во избежании дальнейших неприятностей.
</div>

**Вывод**

Преположительная причина возникновения пропусков: человеческий фактор
Пропуски в зарплатах заменены на значение медианы для соответствующего типа занятости  
Для определения причины пропусков в данных по стажу, надо сначало понять, а чему они вообще соотносятся.  Данные относительно мусорные, было принято волевое решение пропуски заменить на нули, а на остальное забить. в дальнейшем исследовании не понадобятся. так вообще прослеживается что у пенсионеров в данный момент 1000, у молодежи показатели заметно меньше, тут надо гадать что бы это могло значить... Но при этом есть очень крутой разрыв...


### Замена типа данных

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [24]:
df['days_employed'] = df['days_employed'].astype('int')

**Вывод**

Смысла в замене не видел... нужные данные в правильных типах

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Нам же не нужно знать стаж вплоть до дней, или зарплату до копеек. Во-первых, при приведении данные становятся более удобные для чтения. А во-вторых, занимают меньше места в памяти
</div>

### Обработка дубликатов

In [25]:
df[df.duplicated()].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,0,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,покупка жилья для семьи
4182,1,0,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142594.396847,свадьба
4851,0,0,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,свадьба
5557,0,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,сыграть свадьбу
7808,0,0,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514.486412,на проведение свадьбы


In [26]:
df = df.drop_duplicates()

**Вывод**

Удалил полные дубли. считаю что шансов не много, что очень много параметров совпадут, а самое главное зарплата до тысячной. спишем природу этих дубликатов на технические неполадки.
В ручном поиске на данном этапе не вижу смысла искать дубликаты: 21470 значений и шансов что просто люди одного года выпуска и соц. статуса берут кредит на одно и тоже: высокий.

<div class="alert alert-warning">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Здесь важно обратить внимание, что сначала необходимо привести текстовый формат данных к единому виду(регистру), а только потом проверять на повторы. Иначе результат получится дургой. Попробуй вызвать метод sum() и посчитать количество повторов. Да, их немного. Но такие строки могут повлиять на конечный результат
</div>

### Лемматизация

Тут два столбца требующих обработки текста: "Образование" и "Цель кредита". в первом случае должно хватить приведение к нижнему регистру

In [27]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [28]:
df['education'] = df['education'].str.lower()
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [29]:
df['purpose'].unique()  # смотрим и категоризируем данные... 

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Делим цели кредита на следующие категории:  
- Жилье
- Коммерческая недвижимость
- авто 
- образование
- авто 
- иное(вроде нет, но на всякий)  
Тут возникает проблема как поделить неопределенные значение "операции со своей недвижомостью", когда она не определена... Отнесем к коммерческой

In [30]:
# первоначально была попытка сделать с помощью pymystem3... в итоге оказалось что библиотека не оптимизирована под windows
# сделал через то что нашел в google...
import re
import pymorphy2 
morph = pymorphy2.MorphAnalyzer()

In [31]:
def lemmating(row):
    '''function for lemmatize and categore string...

    Args:
        row (str): just string

    Returns:
        str: category of purpose
    '''
    lemmatize_word = morph.parse(row)[0].normal_form
    if 'недвижим' in lemmatize_word:
        if 'жил' in lemmatize_word:
            return 'Жилая недвижимость'
        else:
            return 'Коммерческая недвижимость'
    if 'жил' in lemmatize_word:
        return 'Жилая недвижимость'
    if 'авто' in lemmatize_word:
        return 'Автомобиль'
    if 'свад' in lemmatize_word:
        return 'Свадьба'
    if 'образован' in lemmatize_word or 'обучен' in lemmatize_word or 'учеб' in lemmatize_word:
        return 'Образование'
    return 'Иное'

# Тест работоспособности фукции:
# lemmating('покупка жилая') 

In [32]:
df['purpose_category'] = df['purpose'].apply(lemmating)

In [33]:
df['purpose_category'].value_counts()

Жилая недвижимость           5692
Коммерческая недвижимость    5122
Автомобиль                   4308
Образование                  4014
Свадьба                      2335
Name: purpose_category, dtype: int64

**Вывод**

Категорийные данные приведены в порядок. леммитизированы и разбиты на категории, приведены к нижнему регистру.

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Отлично, лемматизация прошла успешно. Удалось выделить основные цели получения кредита
</div>

### Категоризация данных

Имеет смысл объединить возраста по категориям, а то что-то большой разброс. Т.к. цель исследования: взаимосвязь с семейным положением, то категории сделать не очень крупными, а взять общепринятую классификацию по возрастам молодые – 9-17 лет; ранняя зрелость – 18-24 зрелость – 25-44 года средний пожилой возраст – 45-64 года; ранняя старость – 65-74 года и поздняя старость – 75 лет и старше. Но предварительно надо разобраться с нулями

In [34]:
df_age_zero = df[df['dob_years'] == 0] #смотрим сколько нулей и у кого... может не случайно
df_age_zero

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
99,0,948,0,среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль,Автомобиль
149,0,7,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем,Жилая недвижимость
270,3,5,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью,Жилая недвижимость
578,0,1089,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости,Коммерческая недвижимость
1040,0,3,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль,Автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,0,0,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,жилье,Жилая недвижимость
20462,0,927,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья,Жилая недвижимость
20577,0,908,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость,Коммерческая недвижимость
21179,2,0,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости,Жилая недвижимость


полистав df в jupiter variables: большинство пропусков у женщин... не удивлен. Будем заполнять по среднему значению для данного типа занятости... 

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21471 entries, 0 to 21524
Data columns (total 13 columns):
children            21471 non-null int64
days_employed       21471 non-null int64
dob_years           21471 non-null int64
education           21471 non-null object
education_id        21471 non-null int64
family_status       21471 non-null object
family_status_id    21471 non-null int64
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null int64
total_income        21471 non-null float64
purpose             21471 non-null object
purpose_category    21471 non-null object
dtypes: float64(1), int64(6), object(6)
memory usage: 2.3+ MB


In [36]:
df_mean_age = df.groupby('income_type')['dob_years'].mean() 

In [37]:
df_mean_age = df_mean_age.astype('int')

In [38]:
def check_zero_age(age):
    '''function to check age, if zero, return mean age for this income type

    Args:
        age (int): age

    Returns:
        int: correct age
    '''     
    if age['dob_years'] != 0:
        return age['dob_years']
    return df_mean_age.loc[age['income_type']]

# columns_name = ['income_type', 'dob_years'] Проверка работоспособности функции
# values = [['безработный', 15],['госслужащий', 0]]
# test_func_df = pd.DataFrame(data=values, columns=columns_name)
# test_func_df['dob_years'] = test_func_df.apply(check_zero_age, axis=1)
# test_func_df 


In [39]:
df['dob_years'] = df.apply(check_zero_age, axis=1)

In [40]:
len(df[df['dob_years'] == 0]) # проверил что удалилось

0

In [41]:
def age_category(age):  #функция разбивки на категории
    '''fuction to differencing age people to category, if age < 18 - return None for drop letter

    Args:
        age (int): age of man
    return:
        age_category(str): category of age
    '''
    if age <= 17:
        return 'молодость'
    elif age <= 24:
        return 'ранняя зрелость'
    elif age <= 44:
        return 'зрелость'
    elif age <= 64:
        return 'средний пожилой возраст'
    elif age <= 74:
        return 'ранняя старость'
    else: 
        return 'поздняя старость'
    и
# age_category(75) проверка

In [42]:
df['age_category'] = df['dob_years'].apply(age_category)

In [43]:
df['age_category'].value_counts()

зрелость                   10915
средний пожилой возраст     8782
ранняя старость              897
ранняя зрелость              876
поздняя старость               1
Name: age_category, dtype: int64

In [44]:
df = df[df['age_category'] != 'поздняя старость'] # можно наверно было заменить на ранюю старость... но 1значение... не имеет значения

In [45]:
# проверка что данные категоризированы и нет расхождений. если бы были: надо было бы еще покапаться
display(df.pivot_table(index=['education'], aggfunc='mean', values='education_id'))
display(df.pivot_table(index=['family_status'], aggfunc='mean', values='family_status_id'))

,education_id
education,
высшее,0
начальное,3
неоконченное высшее,2
среднее,1
ученая степень,4


,family_status_id
family_status,
Не женат / не замужем,4
в разводе,3
вдовец / вдова,2
гражданский брак,1
женат / замужем,0


In [46]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [47]:
df_strange_childrens = pd.concat([df[df['children'] == -1], df[df['children'] == 20]]) # создал df со стремными значениями детей
len(df_strange_childrens) # смотрю длину и папаллельно в viewer'е                            

123

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Такие значения можно удалить, либо заменить на более адекватные методом replace()
</div>

In [48]:
df = df[df['children'] != 20] # удаляем, т.к. данных не много и распихивать кажется не корректным
df = df[df['children'] != -1] # тоже самое

In [49]:
def children_category(children):
    '''check children. if person have children return 1, else 0

    Args:
        children (int): sum childrens of person

    Returns:
        int: one or zero
    '''
    if children == 0:
        return 'Нет детей'
    else:
        return 'С детьми'

In [50]:
df['children_availability'] = df['children'].apply(children_category)

In [51]:
def category_purpose(purpose):
    if purpose < 10000:
        return 'A1: Крайняя нищета'
    if purpose < 15000:
        return 'A2: Беднейшие'
    if purpose < 18000:
        return 'A3: Бедные'
    elif purpose < 36000:
        return 'A4: Выше бедности'
    elif purpose < 100000:
        return 'A5: Нижний средний класс'
    elif purpose < 150000:
        return 'A6: Предсредний класс'
    elif purpose < 250000:
        return 'A7: Средний класс'
    elif purpose < 500000:
        return 'A8: Верхний средний класс'
    elif purpose < 1000000:
        return 'A9: Состоятельные'
    else:
        return 'A10: Богатые'

In [52]:
df['category_income'] = df['total_income'].apply(category_purpose)
df['category_income'].value_counts()

A6: Предсредний класс        7125
A7: Средний класс            6981
A5: Нижний средний класс     4369
A8: Верхний средний класс    2575
A9: Состоятельные             197
A4: Выше бедности              75
A10: Богатые                   25
Name: category_income, dtype: int64

т.к довольно мало богатых и выборка будет не показатальна. то объединим богатых,состоятельных с "Верхний средний класс", а также выше бедности и предсредний класс

In [53]:
df['category_income'] = df['category_income'].replace(['A9: Состоятельные', 'A10: Богатые'], 'A8: Верхний средний класс')
df['category_income'] = df['category_income'].replace('A4: Выше бедности', 'A5: Нижний средний класс')
df['category_income'].value_counts()

A6: Предсредний класс        7125
A7: Средний класс            6981
A5: Нижний средний класс     4444
A8: Верхний средний класс    2797
Name: category_income, dtype: int64


<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Очень хорошо, что ты проанализировал данные, нашел статьи, а не взяла числа из головы. На самом деле есть несколько методов разбиения данных на группы. В данном случае можно воспользоваться методом <a href='https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.quantile.html'>quantile()</a> или <a href='https://pandas.pydata.org/docs/reference/api/pandas.qcut.html'>qcut()</a>, в аргументы которого подается необходимое количество групп
 
</div>

**Вывод**

- Данные по возрасту категоризированы с нулями разобрались, никаких сверхъестевственных выводов. разву что Львиная доля клиентов банка 24-64... при этом 24-44 чуть больше чем но 44-64... но пока принципиально это не на что не влияет.    
- Также сделана дополнительно категоризация "есть дети"/"нет детей". 
- Проиведена категоризация данных по зарплате согласно статье [Яндекс Дзен](https://zen.yandex.ru/media/dogecrypto/kak-poschitat-k-kakomu-klassu-vy-otnosites-po-urovniu-dohoda-5b150b5aea0fe700a848e1b1)

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [54]:
display(df.groupby('children')['debt'].agg(['mean', 'count']))
display(df.groupby('children_availability')['debt'].mean()*100)
len(df[df['children'] == 5])

,mean,count
children,,
0,0.075358,14106
1,0.092327,4809
2,0.094542,2052
3,0.081818,330
4,0.097561,41
5,0.000000,9


children_availability
Нет детей    7.535800
С детьми     9.239055
Name: debt, dtype: float64

9

**Вывод**

Наблюдается зависимость надежности клиентов от наличия детей. Однако кол-во детей на надежность не влияет.
Как видим есть зависимость от кол-ва детей и возвратами кредитов... Люди с детьми более чем в 1.6% случаев чаще имеют проблемы с оплатой по кредиту.

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>
Верно, замещики без детей самые ответственные)
    

</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [55]:
data_pivot_family_status = df.pivot_table(index=['family_status'], columns='children_availability', values='debt', aggfunc='mean', margins=True)
data_pivot_family_status

children_availability,Нет детей,С детьми,All
family_status,,,
Не женат / не замужем,0.092838,0.117978,0.097639
в разводе,0.070153,0.071605,0.070648
вдовец / вдова,0.062648,0.096154,0.066316
гражданский брак,0.083546,0.111032,0.092861
женат / замужем,0.069049,0.085750,0.075575
All,0.075358,0.092391,0.081136


**Вывод**

Да. завимость есть... те кто в браке не побывал: кредиты менее охотно возвращают

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>

Посмотрев на сводную таблицу, можно увидеть, что доля просроченых кредитов больше у людей, которые находятся в гражданском браке или не женаты. Чаще всего возращают кредиты вдовцы и вдовы.
    
</div>


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [56]:
data_pivot_income = df.pivot_table(index=['category_income'], columns='children_availability', values='debt', aggfunc='mean', margins=True)
data_pivot_income

children_availability,Нет детей,С детьми,All
category_income,,,
A5: Нижний средний класс,0.072435,0.095477,0.079658
A6: Предсредний класс,0.080128,0.100000,0.086877
A7: Средний класс,0.077211,0.087885,0.080934
A8: Верхний средний класс,0.063193,0.080564,0.069360
All,0.075358,0.092391,0.081136


**Вывод**

Зависимость очень плохо прослеживается. разве что люди с достатком свыше 250т.р. чуть реже имеют проблемы с погашением кредита. Возможно если бы это была основная цель исследования,необходимо было бы сделать более мелкую градацию и уточнить у банка как они производят дифференцируют клиентов по доходу.

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>

Здесь все зависит от того, как ты разделишь данные на группы. Попробуй разделить на равные по количеству заемщиков группы и сравни результат:)
    
</div>


- Как разные цели кредита влияют на его возврат в срок?

In [57]:
data_pivot_purpose = df.pivot_table(index=['purpose_category'], columns='children_availability', values='debt', aggfunc='mean', margins=True)
data_pivot_purpose

children_availability,Нет детей,С детьми,All
purpose_category,,,
Автомобиль,0.085353,0.109484,0.093436
Жилая недвижимость,0.067687,0.074819,0.070129
Коммерческая недвижимость,0.066786,0.091432,0.075201
Образование,0.086677,0.104012,0.092528
Свадьба,0.074675,0.086735,0.078744
All,0.075358,0.092391,0.081136


**Вывод**

основные задолжники: те кто тратиться на обучение и автомобили, а те то с детьми берут на авто: этим вообще не стоит доверять

## Шаг 4. Общий вывод

In [58]:
data_pivot_purpose_and_family = df.pivot_table(index=['purpose_category', 'family_status'], columns='children_availability', values='debt', aggfunc=('mean','count'))
data_pivot_purpose_and_family

count               mean  \
children_availability                           Нет детей С детьми Нет детей   
purpose_category          family_status                                        
Автомобиль                Не женат / не замужем     518.0    117.0  0.123552   
                          в разводе                 195.0     84.0  0.066667   
                          вдовец / вдова            201.0     17.0  0.094527   
                          гражданский брак          282.0    148.0  0.099291   
                          женат / замужем          1651.0   1068.0  0.072078   
Жилая недвижимость        Не женат / не замужем     695.0    156.0  0.069065   
                          в разводе                 237.0    106.0  0.067511   
                          вдовец / вдова            243.0     38.0  0.041152   
                          гражданский брак          330.0    180.0  0.081818   
                          женат / замужем          2218.0   1458.0  0.068079   
Коммерческая недвижимость Не женат / не замужем     592.0    143.0  0.079392   
                          в разводе                 194.0    135.0  0.067010   
                          вдовец / вдова            227.0     27.0  0.052863   
                          гражданский брак          320.0    159.0  0.075000   
                          женат / замужем          2021.0   1275.0  0.063335   
Образование               Не женат / не замужем     457.0    118.0  0.111597   
                          в разводе                 158.0     80.0  0.082278   
                          вдовец / вдова            175.0     22.0  0.068571   
                          гражданский брак          269.0    134.0  0.130112   
                          женат / замужем          1583.0    992.0  0.074542   
Свадьба                   гражданский брак         1540.0    784.0  0.074675   

                                                           
children_availability                            С детьми  
purpose_category          family_status                    
Автомобиль                Не женат / не замужем  0.153846  
                          в разводе              0.095238  
                          вдовец / вдова         0.058824  
                          гражданский брак       0.155405  
                          женат / замужем        0.100187  
Жилая недвижимость        Не женат / не замужем  0.089744  
                          в разводе              0.056604  
                          вдовец / вдова         0.105263  
                          гражданский брак       0.122222  
                          женат / замужем        0.067901  
Коммерческая недвижимость Не женат / не замужем  0.139860  
                          в разводе              0.081481  
                          вдовец / вдова         0.074074  
                          гражданский брак       0.113208  
                          женат / замужем        0.084706  
Образование               Не женат / не замужем  0.093220  
                          в разводе              0.050000  
                          вдовец / вдова         0.136364  
                          гражданский брак       0.186567  
                          женат / замужем        0.097782  
Свадьба                   гражданский брак       0.086735

В результате исспледований обе гипотезы подтвердились:  
- наличие детей отражается на потенциальной надежности клиентов банка. В среднем на 1.5 % люди с детьми имеют проблемы с оплатой кредита
- люди побывавшие в браке более надежные клиенты. 
Также наблюдается отражения надежности клиентов в в завимости от целей кредита. так если цель кредита обучение или какая то сделка с автомобилем: в среднем также на 1.5% чаще возникают проблемы с погашением кредита  
При сведение целей кредита, семейного положения и наличия детей: гипотезы также подтвердились, так в среднем человек с ребенком в гражданском браке берущий кредит 
на образование имеет проблемы в среднем в 18.5% случаев... и наоборот человек в официальном браке берущий кредит на сделку с недвижимостью имеет проблемы только в 6% случаев. Однако стоит отметить что данных для такой выборке маловато и для использования в дальнейшем желательно перепроверить на большем объеме...

<div class="alert alert-success">
<h2> Комментарий от ревьюера<a class="tocSkip"></a></h2>

Поздравляю с успешным  проектом!

- Ты хорошо делаешь выводы и анализируешь результаты.
    
- Код написан аккуратно и качественно, видно, что есть какой-то определенный бэкграунд.

- Соблюдена структура проекта, было приятно проверять:)
    
- Все написано четко и по делу

 

Желаю дальнейших успехов!


</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [-]  заменен вещественный тип данных на целочисленный; *(Не пойму где это надо)*
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.